In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score

In [82]:
lgb1_sub = pd.read_csv('prob/sub_lgb1.csv')
xgb1_sub = pd.read_csv('prob/sub_xgb1.csv')
lgb2_sub = pd.read_csv('prob/sub_lgb2.csv')
xgb2_sub = pd.read_csv('prob/sub_xgb2.csv')

In [83]:
lgb1_oof = pd.read_csv('prob/oof_lgb1.csv')
xgb1_oof = pd.read_csv('prob/oof_xgb1.csv')
lgb2_oof = pd.read_csv('prob/oof_lgb2.csv')
xgb2_oof = pd.read_csv('prob/oof_xgb2.csv')

In [84]:
sub = lgb1_sub.copy()
sub.rename({'prob': 'prob_lgb1'}, axis=1,inplace=True)
sub['prob_xgb1'] = xgb1_sub['prob'].values.tolist()
sub['prob_lgb2'] = lgb2_sub['prob'].values.tolist()
sub['prob_xgb2'] = xgb2_sub['prob'].values.tolist()

sub.head()

,id,prob_lgb1,prob_xgb1,prob_lgb2,prob_xgb2
0,0,0.032812,0.008612,0.025365,0.009897
1,1,0.362178,0.084727,0.251587,0.136121
2,2,0.292020,0.056310,0.042727,0.020272
3,3,0.682772,0.257014,0.434047,0.288397
4,4,0.015556,0.006895,0.015795,0.007014


In [85]:
oof = lgb1_oof.copy()
oof.rename({'prob': 'prob_lgb1'}, axis=1,inplace=True)
oof['prob_xgb1'] = xgb1_oof['prob'].values.tolist()
oof['prob_lgb2'] = lgb2_oof['prob'].values.tolist()
oof['prob_xgb2'] = xgb2_oof['prob'].values.tolist()

oof.head()

,id,prob_lgb1,target,prob_xgb1,prob_lgb2,prob_xgb2
0,0,0.329748,0.0,0.065562,0.0,0.038865
1,1,0.003426,0.0,0.004180,0.0,0.002459
2,2,0.671081,1.0,0.209852,0.0,0.100608
3,3,0.006191,0.0,0.004854,0.0,0.003634
4,4,0.910022,1.0,0.530336,1.0,0.620382


In [86]:
from sklearn.metrics import confusion_matrix
oof['pre'] = 0
for j in range(1,3):
    oof['pre'] = round(oof['prob_lgb'+str(j)])
    con = confusion_matrix(oof['target'], oof['pre'])
    precision = con[0,0]/(con[0,0]+con[1,0])
    recall = con[0,0]/(con[0,0]+con[0,1])
    print('prob_lgb{}_f1 = {}'.format(j,precision*recall/(0.4*precision+0.6*recall)))
from sklearn.metrics import confusion_matrix
oof['pre'] = 0
for j in range(1,3):
    oof['pre'] = round(oof['prob_xgb'+str(j)])
    con = confusion_matrix(oof['target'], oof['pre'])
    precision = con[0,0]/(con[0,0]+con[1,0])
    recall = con[0,0]/(con[0,0]+con[0,1])
    print('prob_xgb{}_f1 = {}'.format(j,precision*recall/(0.4*precision+0.6*recall)))

prob_lgb1_f1 = 0.8996636324507006
prob_lgb2_f1 = 0.9265444893455214
prob_xgb1_f1 = 0.9389615812917594
prob_xgb2_f1 = 0.937595403957703


In [89]:
from sklearn.metrics import precision_score, recall_score
oof['pre'] = 0
label = [0,0,0,0,0]
scoree = [0,0,0,0,0]
for k in range(20,80):
    yuzhi = float(k)/100
    print('______________阈值为{}_______________'.format(yuzhi))
    for j in range(1,3):
        for i in range(oof.shape[0]):
            if (oof['prob_lgb'+str(j)][i] >yuzhi):
                oof['pre'][i] = 1
        precision = precision_score(oof['target'], oof['pre'])
        recall = recall_score(oof['target'], oof['pre'])
        score = (precision * recall) / (0.4 * precision + 0.6 * recall)
        print('prob_lgb{}_f1 = {}'.format(j,score))
        if score > scoree[j]:
            scoree[j] = score
            label[j] = yuzhi
    oof['pre'] = 0
    for j in range(1,3):
        for i in range(oof.shape[0]):
            if (oof['prob_xgb'+str(j)][i] >yuzhi):
                oof['pre'][i] = 1
        precision = precision_score(oof['target'], oof['pre'])
        recall = recall_score(oof['target'], oof['pre'])
        score = (precision * recall) / (0.4 * precision + 0.6 * recall)
        print('prob_xgb{}_f1 = {}'.format(j,score))
        if score > scoree[j+2]:
            scoree[j+2] = score
            label[j+2] = yuzhi

______________阈值为0.2_______________


D:\Users\Huawei\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


prob_lgb1_f1 = 0.2889595103578154
prob_lgb2_f1 = 0.2888079573934837


D:\Users\Huawei\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


prob_xgb1_f1 = 0.4394379844961241
prob_xgb2_f1 = 0.42361462017854146
______________阈值为0.21_______________
prob_lgb1_f1 = 0.2926848746206676
prob_lgb2_f1 = 0.2927335088139108
prob_xgb1_f1 = 0.44296116504854366
prob_xgb2_f1 = 0.4326035346874176
______________阈值为0.22_______________
prob_lgb1_f1 = 0.2964137594535252
prob_lgb2_f1 = 0.2963865205034511
prob_xgb1_f1 = 0.4444092158106109
prob_xgb2_f1 = 0.43492238449526954
______________阈值为0.23_______________
prob_lgb1_f1 = 0.3000911879300339
prob_lgb2_f1 = 0.2998717789634777
prob_xgb1_f1 = 0.44437056737588654
prob_xgb2_f1 = 0.4364053976457077
______________阈值为0.24_______________
prob_lgb1_f1 = 0.3056898159925379
prob_lgb2_f1 = 0.305414551607445
prob_xgb1_f1 = 0.44539736992567175
prob_xgb2_f1 = 0.4389104284720151
______________阈值为0.25_______________
prob_lgb1_f1 = 0.3094990916169219
prob_lgb2_f1 = 0.3090893404080928
prob_xgb1_f1 = 0.4465653352185695
prob_xgb2_f1 = 0.4420825631546519
______________阈值为0.26_______________
prob_lgb1_f1 = 0.312898264

prob_xgb2_f1 = 0.16944665078104318
______________阈值为0.68_______________
prob_lgb1_f1 = 0.4418244611170657
prob_lgb2_f1 = 0.43022820800598577
prob_xgb1_f1 = 0.13190436933223415
prob_xgb2_f1 = 0.1596886741814278
______________阈值为0.69_______________
prob_lgb1_f1 = 0.44867060561299854
prob_lgb2_f1 = 0.4338941165288471
prob_xgb1_f1 = 0.12103505843071784
prob_xgb2_f1 = 0.145578231292517
______________阈值为0.7_______________
prob_lgb1_f1 = 0.4516417529066053
prob_lgb2_f1 = 0.43687451886066203
prob_xgb1_f1 = 0.11654029766919405
prob_xgb2_f1 = 0.13785497656465398
______________阈值为0.71_______________
prob_lgb1_f1 = 0.4398148148148148
prob_lgb2_f1 = 0.43533061857669186
prob_xgb1_f1 = 0.10752688172043011
prob_xgb2_f1 = 0.12520868113522537
______________阈值为0.72_______________
prob_lgb1_f1 = 0.43693322341302554
prob_lgb2_f1 = 0.4365781710914454
prob_xgb1_f1 = 0.09971509971509972
prob_xgb2_f1 = 0.11774600504625737
______________阈值为0.73_______________
prob_lgb1_f1 = 0.43496985357450474
prob_lgb2_f1 = 0.

In [91]:
print(scoree,label)

[0, 0.4516417529066053, 0.43687451886066203, 0.44672281215671916, 0.445906432748538] [0, 0.7, 0.7, 0.26, 0.3]


In [92]:
oof['prob'] = 0.25*(oof['prob_lgb1']*0.5/0.70 +oof['prob_lgb2'] *0.5/0.70 + oof['prob_xgb1'] *0.5/0.26  + oof['prob_xgb2'] *0.5/0.3)
oof['pre'] = round(oof['prob'])
precision = precision_score(oof['target'], oof['pre'])
recall = recall_score(oof['target'], oof['pre'])
score = (precision * recall) / (0.4 * precision + 0.6 * recall)
print('prob_ronghe = {}'.format(score))

prob_ronghe = 0.45072969133410823


In [96]:
sub['prob'] = 0.5*() #+ sub['prob_xgb1'] *0.5/0.26  + sub['prob_xgb2'] *0.5/0.3)
sub.head()

,id,prob_lgb1,prob_xgb1,prob_lgb2,prob_xgb2,prob,target
0,0,0.032812,0.008612,0.025365,0.009897,0.020778,0
1,1,0.362178,0.084727,0.251587,0.136121,0.219202,0
2,2,0.292020,0.056310,0.042727,0.020272,0.119553,0
3,3,0.682772,0.257014,0.434047,0.288397,0.398864,0
4,4,0.015556,0.006895,0.015795,0.007014,0.011197,0


In [97]:
sub['target'] = 0
for i in range(sub.shape[0]):
    if sub.prob[i] >= .5:       # setting threshold to .5
        sub.target[i] = 1
sub.head()

D:\Users\Huawei\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,id,prob_lgb1,prob_xgb1,prob_lgb2,prob_xgb2,prob,target
0,0,0.032812,0.008612,0.025365,0.009897,0.020778,0
1,1,0.362178,0.084727,0.251587,0.136121,0.219202,0
2,2,0.292020,0.056310,0.042727,0.020272,0.119553,0
3,3,0.682772,0.257014,0.434047,0.288397,0.398864,0
4,4,0.015556,0.006895,0.015795,0.007014,0.011197,0


In [98]:
sub[['id', 'target']].to_csv('sub/ronghe.csv', index=False)